In [1]:
import numpy as np
import nltk
import pandas as pd
from datasets import load_dataset
import re
import string
from bs4 import BeautifulSoup
import sklearn
#import spacy

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/alexacole/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexacole/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexacole/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
dataset = load_dataset('artem9k/ai-text-detection-pile')
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'id', 'text'],
        num_rows: 1392522
    })
})

In [3]:
df = pd.DataFrame.from_dict(dataset['train'])
df.head()

,source,id,text
0,human,0,12 Years a Slave: An Analysis of the Film Essa...
1,human,1,20+ Social Media Post Ideas to Radically Simpl...
2,human,2,2022 Russian Invasion of Ukraine in Global Med...
3,human,3,533 U.S. 27 (2001) Kyllo v. United States: The...
4,human,4,A Charles Schwab Corporation Case Essay\n\nCha...


## Reformat Dataset

In [4]:
df['source'].unique()

array(['human', 'ai'], dtype=object)

In [5]:
df['source'] = [1 if x == 'ai' else 0 for x in df['source']]

In [6]:
df.head()

,source,id,text
0,0,0,12 Years a Slave: An Analysis of the Film Essa...
1,0,1,20+ Social Media Post Ideas to Radically Simpl...
2,0,2,2022 Russian Invasion of Ukraine in Global Med...
3,0,3,533 U.S. 27 (2001) Kyllo v. United States: The...
4,0,4,A Charles Schwab Corporation Case Essay\n\nCha...


## Preprocessing

In [7]:
# functions for preprocessing
def remove_urls(text):
    return re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", " ", text) # regex taken from https://www.geeksforgeeks.org/python-check-url-string/

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_extra_whitespace(text):
    text = text.strip()
    text = " ".join(text.split())
    return text

def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words("english")
    tokens = [token for token in tokens if token not in stopwords]
    return " ".join(tokens)

def lemmatizer(text):
    tokens = nltk.word_tokenize(text)
    l = nltk.stem.WordNetLemmatizer()
    tokens = [l.lemmatize(token) for token in tokens]
    return " ".join(tokens)

# eliminate punctuation
def remove_punctuation(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation]
    return " ".join(tokens)

def tokenize_pre_process(text): # for preprocessing using this link: https://spotintelligence.com/2022/12/21/nltk-preprocessing-pipeline/
    # tokenize
    tokens = nltk.word_tokenize(text)

    # remove stop words
    stopwords = nltk.corpus.stopwords.words("english")
    tokens = [token for token in tokens if token not in stopwords]

    # remove top 10% most frequent words 
    fdist = nltk.FreqDist(tokens)
    tokens = [token for token in tokens if fdist[token] < fdist.N() * 0.1]

    # stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # eliminate punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    return tokens

In [8]:
def preprocess_text(text):
    # encoding to ascii
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # convert text to lower case
    text = text.lower()

    # remove html tags 
    text = remove_html(text)

    # remove urls 
    text = remove_urls(text)

    # remove extra whitespace
    text = remove_extra_whitespace(text)

    # remove stop words
    text = remove_stop_words(text)
    
    # remove punctuation
    text = remove_punctuation(text)

    return text

In [9]:
def preprocess_text2(text):
    # encoding to ascii
    text = text.encode('ascii', 'ignore').decode('ascii')
    
    # convert text to lower case
    text = text.lower()

    # remove html tags 
    text = remove_html(text)

    # remove urls 
    text = remove_urls(text)

    # remove extra whitespace
    text = remove_extra_whitespace(text)

    # remove stop words
    text = remove_stop_words(text)
    
    # remove punctuation
    text = remove_punctuation(text)

    # lemmatize words
    text = lemmatizer(text)
    
    return text

## Feature Engineering

### Count Vectorizer

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
vec = CountVectorizer(max_df=0.9,min_df=0.1)
X = vec.fit_transform(df.text[:1000])

In [12]:
vec.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'about', 'above', 'access',
       'according', 'achieve', 'across', 'act', 'action', 'actions',
       'activities', 'activity', 'addition', 'additional', 'additionally',
       'address', 'affect', 'affected', 'affects', 'after', 'against',
       'age', 'al', 'all', 'allow', 'allowed', 'allows', 'almost',
       'already', 'also', 'although', 'always', 'america', 'american',
       'americans', 'among', 'an', 'analysis', 'another', 'any',
       'approach', 'approaches', 'appropriate', 'are', 'area', 'areas',
       'around', 'article', 'aspect', 'aspects', 'associated', 'at',
       'attention', 'author', 'available', 'avoid', 'back', 'based',
       'basis', 'be', 'became', 'because', 'become', 'becomes', 'been',
       'before', 'behavior', 'being', 'believe', 'benefits', 'best',
       'better', 'between', 'black', 'body', 'both', 'business', 'but',
       'by', 'can', 

In [13]:
vec2 = CountVectorizer(preprocessor=preprocess_text,max_df=0.9,min_df=0.1)
X2 = vec2.fit_transform(df.text[:1000])

In [14]:
vec2.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'access', 'according',
       'achieve', 'across', 'act', 'action', 'actions', 'activities',
       'activity', 'addition', 'additional', 'additionally', 'address',
       'affect', 'affected', 'affects', 'age', 'al', 'allow', 'allowed',
       'allows', 'almost', 'already', 'also', 'although', 'always',
       'america', 'american', 'americans', 'among', 'analysis', 'another',
       'approach', 'approaches', 'appropriate', 'area', 'areas', 'around',
       'article', 'aspect', 'aspects', 'associated', 'attention',
       'author', 'authors', 'available', 'avoid', 'back', 'based',
       'basis', 'became', 'become', 'becomes', 'behavior', 'believe',
       'benefits', 'best', 'better', 'body', 'business', 'care', 'case',
       'cases', 'cause', 'caused', 'causes', 'central', 'century',
       'certain', 'challenges', 'change', 'changes', 'characteristics',
       'children',

In [15]:
vec3 = CountVectorizer(preprocessor=preprocess_text2,max_df=0.9,min_df=0.1)
X3 = vec3.fit_transform(df.text[:1000])

In [16]:
vec3.get_feature_names_out()

array(['10', '11', '12', '15', '19', '2017', '2018', '2019', '2020',
       '2021', '2022', 'ability', 'able', 'access', 'according',
       'account', 'achieve', 'across', 'act', 'action', 'activity',
       'addition', 'additional', 'additionally', 'address', 'advantage',
       'affect', 'affected', 'age', 'aim', 'al', 'allow', 'allowed',
       'allows', 'almost', 'already', 'also', 'although', 'always',
       'america', 'american', 'among', 'amount', 'analysis', 'another',
       'application', 'approach', 'appropriate', 'area', 'around',
       'article', 'aspect', 'assessment', 'associated', 'attention',
       'attitude', 'author', 'authority', 'available', 'avoid', 'back',
       'background', 'based', 'basis', 'became', 'become', 'becomes',
       'behavior', 'being', 'belief', 'believe', 'benefit', 'best',
       'better', 'black', 'body', 'book', 'business', 'care', 'case',
       'cause', 'caused', 'center', 'central', 'century', 'certain',
       'challenge', 'chance', '

### TFIDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [18]:
tfidf = TfidfVectorizer(preprocessor=preprocess_text2,max_df=0.9,min_df=0.1) 
tfidf_matrix = tfidf.fit_transform(df.text[:1000]) 

In [19]:
tfidf_matrix

<1000x546 sparse matrix of type '<class 'numpy.float64'>'
	with 104485 stored elements in Compressed Sparse Row format>

In [20]:
tfidf_matrix[0].toarray()

array([[0.        , 0.05350602, 0.37865824, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.13928897, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.05615849, 0.        , 0.10848852,
        0.        , 0.07308116, 0.0830871 , 0.        , 0.        ,
        0.15936191, 0.        , 0.        , 0.03771018, 0.03654908,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.05321945, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.06589735, 0.        , 0.1026759 ,
        0.04175984, 0.        , 0.        , 0.        , 0.05198634,
        0.05133795, 0.        , 0.        , 0.  

### word2vec
#### skipgram

In [59]:
from gensim.models import Word2Vec

In [60]:
# extract training samples and tokenize
training_samples = df['text'][:1000].apply(preprocess_text2)
training_samples = training_samples.apply(nltk.tokenize.word_tokenize)

In [61]:
training_samples

0      [12, year, slave, analysis, film, essay, 2013,...
1      [20+, social, medium, post, idea, radically, s...
2      [2022, russian, invasion, ukraine, global, med...
3      [533, u.s., 27, 2001, kyllo, v., united, state...
4      [charles, schwab, corporation, case, essay, ch...
                             ...                        
995    [working, condition, lead, stress, amazon, rep...
996    [world, medical, relief, covid-19, essay, thro...
997    [world, war, two, ramification, essay, article...
998    [wyeth, christinas, world, vs., vermeer, girl,...
999    [yelling, past, modern, society, essay, yellin...
Name: text, Length: 1000, dtype: object

In [62]:
model1 = Word2Vec(sentences=training_samples, min_count=1,
                                vector_size=200, window=5,sg = 1)

In [63]:
model1

In [64]:
samples_fe = df['text'][:1000].apply(lambda text: np.mean([model1.wv[word] for word in text.split() if word in model1.wv],axis=0))
samples_fe

0      [0.074196376, -0.04712757, -0.10678333, 0.1568...
1      [-0.04162554, 0.06468972, 0.009438879, 0.20073...
2      [0.044293582, 0.016017785, -0.06704636, 0.1293...
3      [0.0015910462, -0.004494369, -0.0040816143, 0....
4      [0.0060147205, 0.07250863, 0.023821475, 0.1515...
                             ...                        
995    [0.005436398, 0.060252234, 0.030332573, 0.1491...
996    [-0.015175242, 0.08629958, 0.069801085, 0.0873...
997    [0.025076255, -0.04014592, -0.06444205, 0.1374...
998    [0.042935327, 0.030128071, -0.07588789, 0.1952...
999    [0.024119573, -0.01209988, -0.065730825, 0.167...
Name: text, Length: 1000, dtype: object

In [65]:
samples_fe[0]

array([ 0.07419638, -0.04712757, -0.10678333,  0.15683906,  0.14515431,
       -0.15465489, -0.19176541,  0.19107528,  0.03810641,  0.22612399,
       -0.03611267, -0.01476813, -0.16460265,  0.15785536,  0.19540825,
       -0.05912674, -0.24772593, -0.17100364,  0.02698568, -0.22694315,
        0.2668589 , -0.125625  ,  0.13611795,  0.10536104,  0.00106118,
       -0.0208538 , -0.07283028,  0.07917844, -0.1586108 ,  0.19942044,
        0.04520908, -0.0897809 , -0.11267109, -0.13570283,  0.13350329,
        0.02113811,  0.22162707,  0.08495472, -0.28973347, -0.0187931 ,
        0.03671626, -0.14193617, -0.12827893,  0.20147258,  0.10764107,
        0.01140182, -0.05461885, -0.03677811,  0.25168172,  0.11662263,
        0.04228579, -0.11505129, -0.15769735,  0.0525364 , -0.062918  ,
        0.16133639, -0.00311779, -0.06461628, -0.40714705,  0.05496838,
       -0.18862246,  0.01941187, -0.11302729,  0.06715795,  0.23720168,
        0.07952263,  0.03036729,  0.15454282, -0.14814423,  0.09

#### CBOW

In [66]:
model2 = Word2Vec(sentences=training_samples, min_count=1,
                                vector_size=200, window=5,sg=0)

In [67]:
samples_fe = df['text'][:1000].apply(lambda text: np.mean([model2.wv[word] for word in text.split() if word in model2.wv],axis=0))
samples_fe

0      [0.13840732, -0.22188556, -0.16674927, 0.21198...
1      [0.024595836, -0.051688176, -0.05450714, 0.417...
2      [0.12491854, -0.1878231, -0.13842414, 0.215766...
3      [0.13495147, -0.13610925, -0.0126397405, 0.166...
4      [0.10271251, -0.08723561, -0.02034141, 0.28603...
                             ...                        
995    [0.03471159, -0.028713746, 0.017016081, 0.2927...
996    [0.031813428, 0.0014286998, 0.056709036, 0.282...
997    [0.09087726, -0.12985303, -0.072528854, 0.1739...
998    [0.07008254, -0.11099894, -0.08870851, 0.27969...
999    [0.07954392, -0.12657845, -0.086049646, 0.2119...
Name: text, Length: 1000, dtype: object

In [68]:
samples_fe[0]

array([ 0.13840732, -0.22188556, -0.16674927,  0.21198055,  0.26666427,
       -0.20671472, -0.3379931 ,  0.6172297 , -0.00439082,  0.48558506,
       -0.14324999, -0.24175769, -0.19305897,  0.4613716 ,  0.02427767,
       -0.22854427, -0.30557477, -0.11403788,  0.06203   , -0.4840235 ,
        0.423883  , -0.35360068,  0.11669571,  0.10242148,  0.09529186,
       -0.27462274, -0.09539352,  0.03002821, -0.42187926,  0.21761674,
        0.2616787 , -0.07417741, -0.02443583, -0.26509362,  0.28882137,
        0.06539727,  0.34661406,  0.15762623, -0.26002583, -0.2097789 ,
       -0.00375025, -0.16684906, -0.10815615,  0.37258038,  0.30269352,
        0.01462196, -0.17952058,  0.02066638,  0.3475885 ,  0.2867722 ,
        0.07128815, -0.09315752, -0.21413389, -0.21104342, -0.07772175,
        0.11919308,  0.13500172, -0.14463179, -0.6000512 , -0.01285411,
       -0.31966248,  0.02187579, -0.20989707,  0.22210233,  0.06149295,
        0.10397222,  0.07188512,  0.47626677, -0.28294414,  0.24

### fasttext

In [69]:
from gensim.models import FastText

In [70]:
model_fasttext = FastText(sentences=training_samples, min_count=1,
                                vector_size=200, window=5)

In [71]:
samples_fe = df['text'][:1000].apply(lambda text: np.mean([model_fasttext.wv[word] for word in text.split() if word in model_fasttext.wv],axis=0))
samples_fe

0      [-0.50773275, 0.058324426, -0.016685331, -0.23...
1      [-0.6261648, 0.22341044, -0.025309082, -0.1853...
2      [-0.5583351, 0.13777289, -0.0191038, -0.214873...
3      [-0.5252659, 0.17695999, -0.023132937, -0.1707...
4      [-0.56707567, 0.22010595, -0.040985283, -0.208...
                             ...                        
995    [-0.5925086, 0.22992893, -0.023463774, -0.1797...
996    [-0.6923324, 0.2846841, -0.02709923, -0.184496...
997    [-0.5767198, 0.124370866, -0.02409606, -0.2200...
998    [-0.60494703, 0.17404887, -0.01755056, -0.1979...
999    [-0.60044765, 0.18412969, -0.020632178, -0.195...
Name: text, Length: 1000, dtype: object

In [72]:
samples_fe[0]

array([-0.50773275,  0.05832443, -0.01668533, -0.23983948,  0.16388766,
       -0.10223313, -0.06961938, -0.25564766, -0.37241256, -0.2716643 ,
       -0.26845917,  0.1034795 , -0.56889915, -0.23011273, -0.20482369,
       -0.24155428,  0.16923608, -0.27766922,  0.2990739 ,  0.2587533 ,
       -0.02013325,  0.12738556, -0.11223251, -0.07805966, -0.26823196,
        0.36440673, -0.22199133,  0.09656067,  0.3537114 , -0.01740418,
        0.07951099, -0.04699205,  0.19276625,  0.06142053,  0.48876423,
        0.61698717, -0.03414697, -0.10478519, -0.0306671 ,  0.17387828,
        0.08842253,  0.27012444, -0.08327803,  0.4866028 , -0.03499604,
       -0.12785476,  0.13906701,  0.0974864 ,  0.1357064 , -0.54331976,
       -0.90102893,  0.12208855,  0.14443061, -0.0648033 ,  0.15065034,
        0.12381642, -0.24882084, -0.13076548,  0.08907124, -0.05459258,
       -0.35284856,  0.26514274, -0.2092277 , -0.07168856, -0.10770483,
        0.38447186,  0.3445882 ,  0.03584883, -0.11163683, -0.50

### GloVe

In [73]:
import gensim.downloader

In [74]:
glove_vectors = gensim.downloader.load('glove-twitter-200')

[=-------------------------------------------------] 3.1% 23.4/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[======--------------------------------------------] 12.8% 97.1/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===========---------------------------------------] 22.7% 172.2/758.5MB downloaded

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=========================================---------] 83.6% 634.4/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==============================================----] 93.8% 711.7/758.5MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [76]:
samples_fe = df['text'][:1000].apply(lambda text: np.mean([glove_vectors[word] for word in text.split() if word in glove_vectors],axis=0))
samples_fe

0      [0.08796369, 0.10185423, 0.07532824, 0.1922056...
1      [0.13417737, 0.14988533, 0.006529619, 0.125828...
2      [0.093451075, 0.14090881, 0.041187413, 0.13554...
3      [0.10934508, 0.14650647, 0.08094985, 0.1336431...
4      [0.14064875, 0.082987, 0.08059668, 0.14689596,...
                             ...                        
995    [0.13349752, 0.052978683, 0.09856825, 0.120453...
996    [0.06676295, 0.0028329766, 0.086873844, 0.1708...
997    [0.0820499, 0.041169856, 0.12953268, 0.1430508...
998    [0.13366817, 0.121703036, 0.1199855, 0.1741929...
999    [0.09286539, 0.074824266, 0.091667704, 0.19394...
Name: text, Length: 1000, dtype: object

In [77]:
samples_fe[0]

array([ 8.79636928e-02,  1.01854227e-01,  7.53282383e-02,  1.92205608e-01,
       -1.37415705e-02,  8.48635882e-02,  3.76222759e-01, -1.37983849e-02,
       -1.07305691e-01, -2.10104257e-01,  3.73063423e-02, -2.93353237e-02,
       -9.10961866e-01, -4.80163097e-02,  1.43521931e-02, -2.03544050e-02,
        3.37492824e-02,  4.11222316e-03, -6.41812757e-02, -1.36727532e-02,
        1.04375824e-01,  9.28258151e-02, -4.31509241e-02, -1.82961021e-02,
       -3.67635973e-02,  8.76572609e-01,  1.41530558e-01,  5.91314435e-02,
        7.09164590e-02,  2.61355881e-02,  5.92161790e-02,  7.34258397e-03,
       -9.62192640e-02, -8.02505091e-02, -2.61530410e-02,  2.86175273e-02,
        4.45067883e-02, -7.76280742e-03,  1.52504668e-01, -1.05464548e-01,
        3.70857447e-01, -9.06854402e-04,  1.07127540e-01, -1.25762112e-02,
        6.68023601e-02, -7.42668211e-02,  1.56346038e-01, -6.78395554e-02,
       -1.68255791e-02, -6.32421896e-02, -9.18123871e-02,  1.93099473e-02,
       -6.35614693e-02, -